In [1]:
print("Hi")

Hi


# Implementing Perceptron

    Perceptron is the simplest type of artificial neural network model

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class Perceptron:
    def __init__(self, input_dim, lr=0.1, epochs=50, random_seed=42):
        np.random.seed(random_seed)
        # weight vector + bias as last element
        self.w = np.random.randn(input_dim + 1) * 0.01
        self.lr = lr
        self.epochs = epochs

    def _add_bias(self, X):
        # append a column of 1s for bias
        return np.hstack([X, np.ones((X.shape[0], 1))])

    def predict_raw(self, X):
        Xb = self._add_bias(X)
        return Xb.dot(self.w)  # linear output

    def predict(self, X):
        # step activation: returns labels 0 or 1
        return (self.predict_raw(X) >= 0).astype(int)

    def fit(self, X, y, verbose=False):
        """
        y must be 0 or 1.
        Perceptron learning rule (online / stochastic update).
        """
        Xb = self._add_bias(X)
        for epoch in range(1, self.epochs + 1):
            errors = 0
            # iterate samples (stochastic)
            for xi, yi in zip(Xb, y):
                activation = xi.dot(self.w)
                pred = 1 if activation >= 0 else 0
                update = self.lr * (yi - pred)
                if update != 0:
                    self.w += update * xi
                    errors += 1
            if verbose:
                acc = np.mean(self.predict(X) == y)
                print(f"Epoch {epoch:2d}  errors={errors:3d}  acc={acc:.3f}")
            # early stop if perfect
            if errors == 0:
                if verbose:
                    print("Training converged.")
                break
        return self


In [5]:
if __name__ == "__main__":
    # --- Demo 1: logical AND ---
    X_and = np.array([[0,0],[0,1],[1,0],[1,1]])
    y_and = np.array([0,0,0,1])

    p = Perceptron(input_dim=2, lr=0.2, epochs=20)
    p.fit(X_and, y_and, verbose=True)
    print("AND predictions:", p.predict(X_and))

    # --- Demo 2: small binary Iris subset (sepal length vs sepal width)
    try:
        from sklearn import datasets
        iris = datasets.load_iris()
        # take only class 0 and 1 (setosa and versicolor) for binary classification
        X = iris.data[iris.target != 2][:, :2]   # first two features for simplicity
        y = iris.target[iris.target != 2]        # labels 0 or 1

        # shuffle
        idx = np.random.permutation(len(y))
        X, y = X[idx], y[idx]

        # split
        split = int(0.7 * len(y))
        X_tr, X_te = X[:split], X[split:]
        y_tr, y_te = y[:split], y[split:]

        p2 = Perceptron(input_dim=2, lr=0.01, epochs=100)
        p2.fit(X_tr, y_tr, verbose=True)
        acc = np.mean(p2.predict(X_te) == y_te)
        print(f"Iris binary test accuracy: {acc:.3f}")
    except Exception as e:
        print("sklearn demo skipped (sklearn not installed). To run Iris demo, install scikit-learn.")

Epoch  1  errors=  2  acc=0.250
Epoch  2  errors=  3  acc=0.500
Epoch  3  errors=  3  acc=0.750
Epoch  4  errors=  2  acc=0.500
Epoch  5  errors=  1  acc=1.000
Epoch  6  errors=  0  acc=1.000
Training converged.
AND predictions: [0 0 0 1]
sklearn demo skipped (sklearn not installed). To run Iris demo, install scikit-learn.
